In [553]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn, optim
import pandas as pd

from tqdm import tqdm

In [554]:
df = pd.read_csv("data\df_01.csv", index_col=[0])

In [555]:
lithology_keys = {30000: 0,
                  65030: 1,
                  65000: 2,
                  80000: 3,
                  74000: 4,
                  70000: 5,
                  70032: 6,
                  88000: 7,
                  86000: 8,
                  99000: 9,
                  90000: 10,
                  93000: 11}

In [556]:
df['FORCE_2020_LITHOFACIES_LITHOLOGY'] = df['FORCE_2020_LITHOFACIES_LITHOLOGY'].map(lithology_keys)

In [557]:
# X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
class CustomDataset(Dataset):

    def __init__(self, dataframe: pd.DataFrame, feature_names: list, target_name: str):
        self.X = torch.tensor(dataframe[feature_names].values, dtype=torch.float32)
        self.X = self.X.reshape(self.X.shape[0], 1, self.X.shape[1])
        self.y = torch.tensor(dataframe[target_name].values, dtype=torch.float32)
        self.y = self.y.type(torch.LongTensor)
        

    def __len__(self) -> int:
        return len(self.y)
    
    def __getitem__(self, index):
        return self.X[index], self.y[index]

In [559]:
feature_names = ['DEPTH_MD', 'X_LOC', 'Y_LOC', 'Z_LOC', 'CALI', 'RMED', 'RDEP', 'RHOB',
                 'GR', 'NPHI', 'PEF', 'DTC', 'BS', 'ROP', 'DTS', 'DCAL', 'DRHO', 'ROPA']

target_name = 'FORCE_2020_LITHOFACIES_LITHOLOGY'

train_dataset = CustomDataset(dataframe=df, feature_names=feature_names, target_name=target_name)
train_data = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True, )

torch.Size([121424, 18])
torch.Size([121424, 1, 18])


In [560]:
len(feature_names)

18

In [561]:
model = nn.Sequential(
    nn.Conv1d(1, 64, 3, padding=1),
    nn.ReLU(),
    nn.Conv1d(64, 64, 3, padding=1),
    nn.ReLU(),
    nn.Conv1d(64, 64, 3, padding=1),
    nn.ReLU(),
    # nn.Conv1d(64, 128, 3, padding=1),
    # nn.ReLU(),
    # nn.Conv1d(128, 128, 3, padding=1),
    # nn.ReLU(),
    # nn.Conv1d(128, 128, 1, padding=1),
    nn.Flatten(),
    nn.Linear(1152, 32),
    nn.ReLU(),
    nn.Linear(32, 12)
)

In [562]:
optimizer = optim.Adam(params=model.parameters(), lr=0.001, weight_decay=0.001)
loss_function = nn.CrossEntropyLoss()

In [563]:
epochs = 5
model.train()

Sequential(
  (0): Conv1d(1, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (1): ReLU()
  (2): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (3): ReLU()
  (4): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (5): ReLU()
  (6): Flatten(start_dim=1, end_dim=-1)
  (7): Linear(in_features=1152, out_features=32, bias=True)
  (8): ReLU()
  (9): Linear(in_features=32, out_features=12, bias=True)
)

In [564]:
for e in range(epochs):
    loss_mean = 0
    lm_count = 0

    train_tqdm = tqdm(train_data, leave=True)
    for x_train, y_train in train_tqdm:
        print(x_train.shape)
        predict = model(x_train)
        loss = loss_function(predict, y_train)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        lm_count += 1
        loss_mean = 1/lm_count * loss.item() + (1 - 1/lm_count) * loss_mean
        train_tqdm.set_description(f'Epoch [{e+1}/{epochs}], loss_mean={loss_mean:.3f} ')

  0%|          | 0/1898 [00:00<?, ?it/s]

torch.Size([64, 1, 18])


Epoch [1/5], loss_mean=21237.221 :   0%|          | 3/1898 [00:00<06:15,  5.04it/s]

torch.Size([64, 1, 18])
torch.Size([64, 1, 18])


Epoch [1/5], loss_mean=18347.542 :   0%|          | 8/1898 [00:00<02:14, 14.03it/s]

...

torch.Size([64, 1, 18])
torch.Size([64, 1, 18])


Epoch [5/5], loss_mean=1.492 : 100%|█████████▉| 1896/1898 [04:16<00:00,  7.24it/s]

torch.Size([64, 1, 18])


Epoch [5/5], loss_mean=1.492 : 100%|██████████| 1898/1898 [04:16<00:00,  7.40it/s]


torch.Size([16, 1, 18])


In [565]:
class ConvNet(nn.Module):
    def __init__(self, n_class, in_channels):
        super().__init__()

        self.conv_1 = nn.Conv1d(in_channels=in_channels, out_channels=64, kernel_size=2, padding=1)
        self.conv_2 = nn.Conv1d(in_channels=64, out_channels=64, kernel_size=2, padding=1)
